In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
import sklearn 
import random

Using TensorFlow backend.


In [2]:
def seq(length, n_features):
    return np.array([random.randint(0, n_features-1) for i in range(length)])

LENGTH = 10
FEATS = 10

seqX = seq(LENGTH, FEATS)

In [3]:
def one_hot_encode_hard(sequence, n_features):
  encoding = list()
  for value in sequence:
    vector = [0 for _ in range(n_features)]
    vector[value] = 1
    encoding.append(vector)
  return np.array(encoding)


def one_hot_decode_hard(encoded_seq):
  return [np.argmax(vector) for vector in encoded_seq]

In [4]:
hard_1_hot = one_hot_encode_hard(seqX, FEATS)

In [5]:
# Generate small example for an LSTM network
def generate_example(length, n_features, out_index):
    sequence = seq(length, n_features)
    encoded = one_hot_encode_hard(sequence, n_features)
    X = encoded.reshape((1, length, n_features))
    y = encoded[out_index].reshape(1, n_features)
    return X, y

X, y = generate_example(25, 100, 2)
print(X.shape)
print(y.shape)

(1, 25, 100)
(1, 100)


In [6]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

# Define our vanilla LSTM model.
length = 5
n_features = 10
out_index = 4
model = Sequential()
model.add(LSTM(25, input_shape=(length, n_features)))
model.add(Dense(n_features, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25)                3600      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                260       
Total params: 3,860
Trainable params: 3,860
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Train our model.
# The range denotes the epochs in the general fit function and each generated example is a batch
# Each epoch therefore runs through our batch 1 time and the weights are updated.
for i in range(1000):
  X, y = generate_example(length, n_features, out_index)
  model.fit(X, y, epochs=1, verbose=0)

In [8]:
# Evaluate our model
correct = 0
for i in range(100):
    X, y = generate_example(length, n_features, out_index)
    yhat = model.predict_classes(X)
    if yhat == one_hot_decode_hard(y):
        correct += 1
        
print('Accuracy: %f' % ((correct/100)*100.0))

Accuracy: 100.000000


In [9]:
# Prediction on a new sequence. 
X, y = generate_example(length, n_features, out_index) 
yhat = model.predict(X)
print('Sequence: %s' % [one_hot_decode_hard(x) for x in X]) 
print('Expected: %s' % one_hot_decode_hard(y)) 
print('Predicted: %s' % one_hot_decode_hard(yhat))

Sequence: [[5, 6, 9, 8, 0]]
Expected: [0]
Predicted: [0]
